In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import pytesseract
from PIL import Image

In [3]:
img = Image.open('photo.jpeg')
img = img.convert('L')

In [4]:
# Crop extra part
def crop1(img):
    left, top, right,bottom = 95,91,665,745
    cropped = img.crop((left,top,right,bottom))
    return cropped

In [5]:
# Split into seven days
def crop2(img):
    # starting from monday
    daysList = [[0,0,570,96], # Monday
                [0,97,570,189], #Tuesday
                [0,190,570,282], # Wednesday
                [0,283,570,375], # Thurshday
                [0,376,570,468], #Friday
                [0,469,570,561], #Saturday
                [0,562,570,654], #Sunday
               ]
    
    def helperFunc(l,index):
        left, top, right,bottom = l[index][0],l[index][1],l[index][2],l[index][3]
        cropped = img.crop((left,top,right,bottom))
        return cropped
    
    finalList = []
    for i in range(7):
        finalList.append(helperFunc(daysList,i))
    return finalList



In [6]:
# return final list for periods after cropping
def getList(l):
    
    def helperFunc(img):
        left1, top1, right1, bottom1 = 0,0,95,30 # for 1st period
        left2, top2, right2, bottom2 = 0,30,95,60 # for 2nd period
        left3, top3, right3, bottom3 = 0,60,95,90 # for 3rd period
        return [img.crop((left1, top1, right1, bottom1)),
               img.crop((left2, top2, right2, bottom2)),
               img.crop((left3, top3, right3, bottom3))]
    
    finalList = []
    for i in range(7):
        if i == 2: # For wednesday
            continue
        periods = helperFunc(l[i])
        p1 = pytesseract.image_to_string(periods[0],config=' --psm 11')
        p2 = pytesseract.image_to_string(periods[1],config=' --psm 11')
        p3 = pytesseract.image_to_string(periods[2],config=' --psm 11')
        
        finalList.append([[p1.lower()[0],periods[0]],
                          [p2.lower()[0],periods[1]],
                          [p3.lower()[0],periods[2]]])
    return finalList

In [7]:
# Correcting not recognised using help of user    
def correct(l):
    
    def helperFunc(img): # asks user for value
            from threading import Thread
            thr = Thread(target = display, args=(img,))
            thr.start()
            string = "Press 1 for physics,2 for chemistry, 3 for maths"
            inp = int(input(string))
            return inp
    
    # correcting values from users
    for i in l:
        for j in range(3):
            if i[j][0]!= ("p" or "c" or "m"):
                print(i[j][0])
                value = helperFunc(i[j][1])
                if (value == 1):i[j][0] = "p"
                elif (value == 2):i[j][0] = "c"
                elif (value == 3):i[j][0] = "m"
                print(i[j][0])  
                
                
    # removing pictures from list
    for i in l:
        for j in range(3):
            i[j] = i[j][0]
    return l
            
            



In [11]:

l = crop2(crop1(img))
# display(l[0],l[1] , l[2],l[3],l[4],l[5],l[6])    
listL = correct(getList(l))

i


Press 1 for physics,2 for chemistry, 3 for maths2
c
m


Press 1 for physics,2 for chemistry, 3 for maths3
m
c


Press 1 for physics,2 for chemistry, 3 for maths2
c
m


Press 1 for physics,2 for chemistry, 3 for maths3
m
i


...

Press 1 for physics,2 for chemistry, 3 for maths2
c
m


Press 1 for physics,2 for chemistry, 3 for maths3
m
m


Press 1 for physics,2 for chemistry, 3 for maths3
m
i


Press 1 for physics,2 for chemistry, 3 for maths1
p
|


Press 1 for physics,2 for chemistry, 3 for maths1
p
[['c', 'm', 'p'], ['c', 'm', 'p'], ['c', 'p', 'm'], ['m', 'c', 'p'], ['p', 'c', 'm'], ['m', 'p', 'p']]


In [15]:
 listL = [['c', 'm', 'p'], ['c', 'm', 'p'], ['c', 'p', 'm'], ['m', 'c', 'p'], ['p', 'c', 'm'], ['m', 'p', 'p']]

## Operations on list 

In [16]:
from datetime import timedelta
import datetime
# assuming pic uploaded on sunday

In [17]:
# Helper Functions
def createDates(dayNo):
    reqDate = datetime.datetime.now() + timedelta(days=dayNo)
    return [reqDate.year,reqDate.month,reqDate.day]

def createTimeDate(dayNo,eventNo):
    timeDict = {0:[[16,0,0],[17,15,0]],1:[[17,30,0],[18,45,0]],2:[[19,0,0],[20,15,0]]}
    reqFormatList  = createDates(dayNo)
    reqFormatList.append(timeDict[eventNo])
    TimeZone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
    startTime = datetime.datetime(reqFormatList[0],reqFormatList[1],reqFormatList[2],reqFormatList[3][0][0],reqFormatList[3][0][1],reqFormatList[3][0][2],tzinfo=TimeZone)
    endTime = datetime.datetime(reqFormatList[0],reqFormatList[1],reqFormatList[2],reqFormatList[3][1][0],reqFormatList[3][1][1],reqFormatList[3][1][2],tzinfo=TimeZone)
    
    return startTime.strftime("%Y-%m-%dT%H:%M:%S") , endTime.strftime("%Y-%m-%dT%H:%M:%S")

# main Func
def create_event(dayNo,eventNo,event):
    eventDicti = {"p":"Physics Class","c":"Chemistry Class","m":"Maths Class"}
    startTime , endTime = createTimeDate(dayNo,eventNo)
    return {"summary":eventDicti[event],"start":{"dateTime":startTime,"timeZone":"Asia/Kolkata"},"end":{"dateTime":endTime,"timeZone":"Asia/Kolkata"}}
  
   
    
    

def create_eventStrings(l):
    listi = []
    vari = 0
    for i in range(1,8):# DayNO --> 1-monday,2-tuesday and so on
        if i == 3:# Wednesday
            continue
        for j in range(3):
           listi.append(create_event(i,j,l[vari][j]) )
        vari+=1
    return listi

print(create_eventStrings(listL))



[{'summary': 'Chemistry Class', 'start': {'dateTime': '2021-11-22T16:00:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2021-11-22T17:15:00', 'timeZone': 'Asia/Kolkata'}}, {'summary': 'Maths Class', 'start': {'dateTime': '2021-11-22T17:30:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2021-11-22T18:45:00', 'timeZone': 'Asia/Kolkata'}}, {'summary': 'Physics Class', 'start': {'dateTime': '2021-11-22T19:00:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2021-11-22T20:15:00', 'timeZone': 'Asia/Kolkata'}}, {'summary': 'Chemistry Class', 'start': {'dateTime': '2021-11-23T16:00:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2021-11-23T17:15:00', 'timeZone': 'Asia/Kolkata'}}, {'summary': 'Maths Class', 'start': {'dateTime': '2021-11-23T17:30:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTime': '2021-11-23T18:45:00', 'timeZone': 'Asia/Kolkata'}}, {'summary': 'Physics Class', 'start': {'dateTime': '2021-11-23T19:00:00', 'timeZone': 'Asia/Kolkata'}, 'end': {'dateTim

In [18]:
from __future__ import print_function
def buildService():
    
    import datetime
    import os.path
    from googleapiclient.discovery import build
    from google_auth_oauthlib.flow import InstalledAppFlow
    from google.auth.transport.requests import Request
    from google.oauth2.credentials import Credentials
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

   
service = buildService()
eventStrings = create_eventStrings(listL)
for i in (eventStrings):
    event = service.events().insert(calendarId='primary', body=i).execute()
    print ('Event created: %s' % (event.get('htmlLink')))

    
    





Event created: https://www.google.com/calendar/event?eid=ZGVjbGVlaDh2NGttNHZscGlnN2w2bDE5M2sgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=Yzl2N2hjZXNob2tqZWhzcGcyMmswbm5yaWsgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=bmk2Y3E0azhyaDMwYWZnOTVqY3Q5YWJlbHMgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=Y244NDlpYzI0dWhiYjljdGU5ZGxuMnI4MjAgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=MjRhaGF1MzZjazJnOWkxOHVoYXNhajk0ZW8gdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=ajhkc2hvZzk5Z2lqNnYwYTZ2b3JmY2RsYzAgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=MXA0Z3R2ZGZuaXB2MGZzbW5iOGliNnA4YXMgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=NmY4cWRzc3MzNWZ0anJjcXQ0Y21sbGRqODQgdGFubWF5MDZkYWdhQG0
Event created: https://www.google.com/calendar/event?eid=MGp2ZDBydmU0amlzOHI4ZmY0bmhxb2U4NjggdGF